Libraries

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

Data Preprocessing

In [12]:
# Import training set
train_path = 'D:/Onedrive/experiments/experiments/ML\DL/P16-Deep-Learning-AZ/Deep Learning A-Z/Part 3 - Recurrent Neural Networks (RNN)/Google_Stock_Price_Train.csv'
dataset_train = pd.read_csv(train_path)
training_set = dataset_train.iloc[:, 1:2].values # only the 'open' column

# Feature Scaling
# its recommended to use normalization when sigmoid is used as activation function
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 60 time steps and 1 output
# basically the last 60 time steps of data will be used to predict the next output
x_train = [] # last 60 stock prices
y_train = [] # next output
# also means we have to start predicting from 60th day onwards and not from day 1
for i in range(60, 1258):
    x_train.append(training_set_scaled[i - 60: i, 0])
    y_train.append(training_set_scaled[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshaping the data to include more params later (if required)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


Building the RNN

Making Predictions and Visualization